In [30]:
import pandas as pd
from util.readme_parser import ReadmeSectionParser
from e2e_system.reproder import HierarchicalReproder, _ground_truth
from pathlib import Path

import os
from tqdm.auto import tqdm
from datetime import datetime
import shutil
from util.file_helper import get_readme_with_repo_name

import warnings
warnings.filterwarnings("ignore")


In [31]:
keys = ["header", "parent_header", "content"]
model = "hierarchical"
model_path = "model/hierarchical\\autolabelled2"
neurips_df = pd.read_csv(
    'data/paperswithcode/neurips_checklist_manuel-100.csv')
neurips_df["score"] = neurips_df.select_dtypes(bool).sum(axis=1)


In [32]:
def evaluate(reproder: HierarchicalReproder, m_df, type=1):
    df = m_df.copy()
    for repo in (pbar := tqdm(neurips_df.url.values, desc="Evaluating")):
        pbar.set_postfix_str(f"Evaluating {repo}")
        readme = get_readme_with_repo_name("neurips", repo)
        sections = ReadmeSectionParser(
            repo, readme).parse_sections(group_by_parent=False)
        record = df[df["url"] == repo]
        if len(sections) == 0:
            df.loc[record.index, "reprod_score"] = 0
            continue
        readme_text = ReadmeSectionParser.merge_sections(
            pd.DataFrame(sections), with_newline=True, keys=keys)
        logits = reproder.evaluate(readme_text)
        reprod_score = reproder.calculate_reproducibility(logits, type=type)
        record = df[df["url"] == repo]
        df.loc[record.index, "reprod_score"] = reprod_score
    return df


In [33]:

path = Path(model_path)


In [34]:
path


WindowsPath('model/hierarchical/autolabelled2')

In [35]:
reproder = HierarchicalReproder(path, keys)
direct_classification = evaluate(reproder, neurips_df, type=1)
# weighted_score = evaluate(reproder, neurips_df, type=2)
# direct_coeff = evaluate(reproder, neurips_df, type=3)


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

In [36]:
direct_classification


,url,stars,introduction,requirements,pretrained_model,evaluation,training,results,score,reprod_score
0,deepmind/lab,6082,True,True,False,True,True,False,4,1.0
1,zihangdai/xlnet,5114,True,False,True,True,True,True,5,3.0
2,tensorflow/tpu,3120,True,False,False,False,False,False,1,1.0
3,tensorflow/lingvo,1914,True,True,False,False,True,False,3,3.0
4,facebookresearch/XLM,1913,True,True,True,True,True,True,6,3.0
...,...,...,...,...,...,...,...,...,...,...
95,fkunstner/limitations-empirical-fisher,26,True,True,False,True,False,False,3,1.0
96,max-andr/provably-robust-boosting,26,True,True,False,True,True,False,4,3.0
97,PwnerHarry/Stronger_GCN,26,True,True,False,False,True,True,4,3.0
98,PKU-AI-Edge/FEN,23,True,False,False,False,False,False,1,1.0


In [37]:
os.makedirs(
    "data/paperswithcode/evaluated\hierarchical/autolabelled2", exist_ok=True)
direct_classification.to_csv(
    f"data/paperswithcode/evaluated\hierarchical/autolabelled2/direct_classification.csv", index=False)
# weighted_score.to_csv(
#     f"data/paperswithcode/evaluated\hierarchical/autolabelled2/weighted_score.csv", index=False)
# direct_coeff.to_csv(
#     f"data/paperswithcode/evaluated\hierarchical/autolabelled2/direct_coeff.csv", index=False)


In [38]:
# save_main_dir = f'data/paperswithcode/evaluated/hierarchical/{model}/'
# save_dir = os.path.join(save_main_dir, model.split("\\")[-1])
# shutil.rmtree(save_dir, ignore_errors=True)
# os.makedirs(save_dir, exist_ok=True)
# info = {
#     "model_dir": model,
#     "keys": keys,
#     "time": datetime.now().strftime("%Y%m%d-%H%M%S")
# }
# # open(os.path.join(save_dir, "info.json"), 'w').write(
# #     json.dumps(info, indent=4))

# # direct_classification.to_csv(
# #     f'{save_dir}/direct_classification.csv', index=False)
# # weighted_score.to_csv(
# #     f'{save_dir}/weighted_score.csv', index=False)
# # direct_coeff.to_csv(
# #     f'{save_dir}/direct_coeff.csv', index=False)
